In [29]:
from datetime import datetime, timedelta
import sqlite3
#import xlsxwriter
import xlwings as xw
import openpyxl
from openpyxl import *
from openpyxl.styles import *
import os

In [30]:
def round_to_quarter_hour(minutes):
    return round(minutes / 15) * 15

In [31]:
def convert_minutes_to_hours(minutes):
    return minutes / 60

In [32]:
def is_saturday(date_str):
    # Parse the date string into a datetime object
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    # Check if the day of the week is Saturday (5)
    return date_obj.weekday() == 5

In [33]:
def get_day_of_week(date_str):
    # Parse the date string into a datetime object
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    # Get the day of the week as a full name (e.g., "Monday", "Tuesday")
    day_of_week = date_obj.strftime('%A')
    return day_of_week

In [34]:
def createReportWorkbook(new_wb, template_sheet_name):
    # Load the existing workbook
    template_path = 'Template Sheets - Copy.xlsx'
    # Copy the template sheet to the new workbook
    try:
        excel_app = xw.App(visible=False)
        wb = excel_app.books.open(template_path)
        for sheet in wb.sheets:
            if sheet.name == template_sheet_name:
                sheet.api.Copy()
                wb_new = xw.books.active
                wb_new.save(new_wb)
                wb_new.close()
    finally:
        excel_app.quit()

In [35]:
def getKey(e):
    dat1, dat2 = e
    return datetime.strptime(dat1[2], "%Y-%m-%d %H:%M:%S")

In [58]:
def setCell(cell, newValue, newFont, newNumberFormat, newBorder, newAlignment):
    cell.value = newValue
    cell.font = newFont
    cell.number_format = newNumberFormat
    cell.border = newBorder
    cell.alignment = newAlignment

In [111]:
# This method takes the punch data for one employee and formats the given openpyxl sheet with the data
def reportTimecardData(
    sheet: openpyxl.worksheet,
    data: list
):
    #last row used so that we can set the print_area correctly and will be returned at the end of the method
    lastRow = 4
    #sort the data in order by punch_in datetime (chronilogical order) then store it in a dictionary
    data.sort(key=getKey)
    databyDate = {}
    for tup in data:
        dataDate = datetime.strptime(tup[0][2], '%Y-%m-%d %H:%M:%S').date()
        if dataDate not in databyDate.keys():
            databyDate[dataDate] = [tup]
        else:
            dateLst = databyDate[dataDate]
            dateLst.append(tup)
            databyDate[dataDate] = dateLst
    #totals
    TotalConstructionTime = TotalServiceTime = 0
    ##all of my default styling with fonts, number formats, borders, and alignment
    normal_font, bold_font = Font(name='Arial', size=10, bold=False), Font(name='Arial', size=10, bold=True)
    date_number_format, hrs_number_format, time_duration_format, time_format = 'mm-dd-yy', '0.00 "hrs"', 'h:mm:ss', 'hh:mm AM/PM'
    workTime_border = Border( top=Side(border_style='thin', color='00000000'), bottom=Side(border_style='thin', color='00000000') )
    rightAlign, centerAlign, leftAlign = Alignment(horizontal='right'), Alignment(horizontal='center'), Alignment(horizontal='left')
    #time to get data onto the page
    for date in databyDate.keys():
        TotalingRow = lastRow
        TotalTime = TotOfficeTime = TotShopTime = TotLunchTime = 0
        allPunchesOnDate = databyDate[date]
        for punch, work in allPunchesOnDate:
            punch_in = datetime.strptime(punch[2], '%Y-%m-%d %H:%M:%S')
            punch_out = datetime.strptime(punch[3], '%Y-%m-%d %H:%M:%S') if punch[3] else datetime.now()
            shift_duration = convert_minutes_to_hours(round_to_quarter_hour((punch_out - punch_in).total_seconds() / 60))
            lunch = 0.5 if not punch[6] and shift_duration >= 6.0 else 0
            constructionLst = []
            serviceLst = []
            TotConst = TotSer = TotOfc = 0
            if work:
                for worktype, cust, time in work:
                    hrs = time / 60
                    if worktype == "Office":
                        TotOfc = TotOfc + hrs
                        TotOfficeTime = TotOfficeTime + hrs
                    elif worktype == "Service":
                        TotSer = TotSer + hrs
                        serviceLst.append((cust, hrs))
                    else:
                        TotConst = TotConst + hrs
                        constructionLst.append((cust, hrs))
            #start inserting punch data into sheet
            lastRow = lastRow + 1
            setCell(sheet[f"I{lastRow}"], "Clock-In", bold_font, 'General', Border(), Alignment())
            setCell(sheet[f"J{lastRow}"], punch_in.time(), normal_font, time_format, Border(), leftAlign)
            if len(constructionLst) > 0:
                lastRow = lastRow + 1
                setCell(sheet[f"J{lastRow}"], "Construction", bold_font, 'General', workTime_border, rightAlign)
                setCell(sheet[f"K{lastRow}"], TotConst, normal_font, hrs_number_format, workTime_border, Alignment())
                TotalConstructionTime = TotalConstructionTime + TotConst
                for name, hrs in constructionLst:
                    lastRow = lastRow + 1
                    setCell(sheet[f"J{lastRow}"], name, normal_font, 'General', Border(), rightAlign)
                    setCell(sheet[f"K{lastRow}"], hrs, normal_font, hrs_number_format, Border(), Alignment())
            if len(serviceLst) > 0:
                lastRow = lastRow + 1
                setCell(sheet[f"J{lastRow}"], "Service", bold_font, 'General', workTime_border, rightAlign)
                setCell(sheet[f"K{lastRow}"], TotSer, normal_font, hrs_number_format, workTime_border, Alignment())
                TotalServiceTime = TotalServiceTime + TotSer
                for name, hrs in serviceLst:
                    lastRow = lastRow + 1
                    setCell(sheet[f"J{lastRow}"], name, normal_font, 'General', Border(), rightAlign)
                    setCell(sheet[f"K{lastRow}"], hrs, normal_font, hrs_number_format, Border(), Alignment())
            lastRow = lastRow + 1
            setCell(sheet[f"I{lastRow}"], "Clock-Out", bold_font, 'General', Border(), Alignment())
            setCell(sheet[f"J{lastRow}"], punch_out.time(), normal_font, time_format, Border(), leftAlign)
            lastRow = lastRow + 1
            setCell(sheet[f"K{lastRow}"], "Elapsed Time:", bold_font, 'General', Border(), rightAlign)
            setCell(sheet[f"L{lastRow}"], shift_duration, normal_font, hrs_number_format, Border(), leftAlign)
            TotShopTime = TotShopTime + (shift_duration - lunch - TotConst - TotSer - TotOfc)
            TotalTime = TotalTime + shift_duration
            TotLunchTime = TotLunchTime + lunch
        #total all of the punches on the day
        setCell(sheet[f"H{TotalingRow}"], date.strftime('%A'), bold_font, 'General', Border(), Alignment())
        setCell(sheet[f"I{TotalingRow}"], date, normal_font, date_number_format, Border(), Alignment())
        setCell(sheet[f"J{TotalingRow}"], "Total:", bold_font, 'General', Border(), rightAlign)
        setCell(sheet[f"K{TotalingRow}"], TotalTime, normal_font, hrs_number_format, Border(), Alignment())
        setCell(sheet[f"L{TotalingRow}"], "Shop:", bold_font, 'General', Border(), rightAlign)
        setCell(sheet[f"M{TotalingRow}"], TotShopTime, normal_font, hrs_number_format, Border(), Alignment())
        setCell(sheet[f"N{TotalingRow}"], "Lunch:", bold_font, 'General', Border(), rightAlign)
        setCell(sheet[f"O{TotalingRow}"], TotLunchTime, normal_font, hrs_number_format, Border(), Alignment())
        setCell(sheet[f"P{TotalingRow}"], "Office:", bold_font, 'General', Border(), rightAlign)
        setCell(sheet[f"Q{TotalingRow}"], TotOfficeTime, normal_font, hrs_number_format, Border(), Alignment())
        setCell(sheet[f"R{TotalingRow}"], "Paid Hrs:", bold_font, 'General', Border(), rightAlign)
        setCell(sheet[f"S{TotalingRow}"], f"=K{TotalingRow}-O{TotalingRow}", normal_font, hrs_number_format, Border(), Alignment())
        lastRow = lastRow + 2
    sheet["C5"].value = TotalConstructionTime
    sheet["D5"].value = TotalServiceTime
    return lastRow

In [134]:
def weekly_report(
    db: str,
    week_end_date: str,
    employee_group: str
):
    debug = False
    try:
        # Parse the provided date
        EOW = datetime.strptime(week_end_date, "%Y-%m-%d")
        week_start = EOW - timedelta(days=6)
        if not is_saturday(week_end_date):
            print(f"{week_end_date} is not a saturday it is a: {get_day_of_week(week_end_date)}")
            return
        week_end = EOW + timedelta(days=1)

        # Connect to the database
        conn = sqlite3.connect(db)
        if debug:
            conn.set_trace_callback(print)
        cursor = conn.cursor()
        
        # Fetch the group ID from the employee_group table
        cursor.execute("SELECT id FROM employee_group WHERE name = ?", (employee_group,))
        group_id = cursor.fetchone()
        if not group_id:
            print(f"Employee group '{employee_group}' not found.")
            return
        group_id = group_id[0]

        # Fetch the employee IDs for the selected group
        cursor.execute("SELECT employeeID FROM group_member WHERE groupID = ?", (group_id,))
        employee_ids = [row[0] for row in cursor.fetchall()]

        if not employee_ids:
            print(f"No employees found in group '{employee_group}'.")
            return

        # Prepare SQL query to fetch all punches for the previous week for the employees in the group
        placeholders = ",".join("?" for _ in employee_ids)
        sql_query = f"""
            SELECT 
                e.id,
                e.name,
                pc.id,
                pc.punchInTime,
                pc.punchOutTime,
                pc.punchInApproval,
                pc.punchOutApproval,
                pc.ignoreLunchBreak
            FROM punch_clock pc
            JOIN employee e ON pc.employeeID = e.id
            WHERE pc.punchInTime BETWEEN ? AND ? AND e.id IN ({placeholders})
            ORDER BY e.name, pc.punchInTime
        """

        params = [week_start, week_end] + employee_ids

        # Execute the query
        cursor.execute(sql_query, params)
        punches = cursor.fetchall()

        # Prepare to fetch work punches
        punch_data = {}
        employee_data = {}
        for punch in punches:
            employeeID, name, punch_id, punch_in, punch_out, in_approval, out_approval, ignore_lunch = punch
            punch = (name, punch_id, punch_in, punch_out, in_approval, out_approval, ignore_lunch)
            #Fetch work punches for each punch clock entry
            cursor.execute("""
                SELECT
                    wt.punchType,
                    c.name,
                    wt.timeSpent
                FROM work_time wt
                JOIN customer c ON wt.customerID = c.id
                WHERE wt.punchID = ?
                ORDER BY wt.timeStarted
            """, (punch_id,))
            work_punches = cursor.fetchall()
            
            if name not in punch_data:
                punch_data[name] = []
                cursor.execute("""
                    SELECT
                        name,
                        addressLine1,
                        addressLine2,
                        addressCity,
                        addressState,
                        addressZip,
                        phoneNumber
                    FROM employee
                    WHERE id = ?
                """, (employeeID,))
                employee = cursor.fetchall()
                employee_data[name] = employee[0]
            punch_data[name].append((punch, work_punches))

        conn.close()

        #if debugging print out the timeclock data
        if debug:
            for key, data in punch_data.items():
                print(f"=-=-=-=-=-=-\n{key}")
                for dat1, dat2 in data: #data is a list of tuples that has dat1 
                    print(f"    {dat1}")   #which has a tuple of all the information 
                    print(f"        {dat2}") #for the shift punch and dat2 is a list of workpunches for the shift
            
        
        if not punches:
            print(f"No punches found for the week ending on {week_end_date}.")
            return

        # Create an Excel file
        file_path = f"Weekly_Report_{week_end_date}.xlsx"
        createReportWorkbook(file_path, "Timecard")
        # Edit Excel File
        employees = [key for key in punch_data.keys()]
        # Create all the worksheets
        wb = load_workbook(file_path)
        ws = wb["Timecard"]
        ws.title = employees[0]
        # Create several copies of the template sheet within the new workbook
        for e in employees[1::]:
            # Copy the contentwb from the original new sheet to the new copy
            source = wb.active
            target = wb.copy_worksheet(source)
            target.title = e
        for e in employees:
            sheet = wb[e]
            sheet["D8"].value = week_end_date
            sheet["D11"].value = f"{employee_data[e][0]}"
            sheet["D14"].value = f"{employee_data[e][1]} {employee_data[e][2]}"
            sheet["D15"].value = f"{employee_data[e][3]}, {employee_data[e][4]} {employee_data[e][5]}"
            sheet["D18"].value = f"{employee_data[e][6]}"
            lastRow = reportTimecardData(sheet, punch_data[e])
            if lastRow < 1:
                print(f"A critical error/bug occured in reportTimecardData on {e} because offset returned was {lastRow}!")
            else:
                sheet.print_area = f"A1:G21 H1:S{lastRow}"
        wb.save(file_path)
        
        
        # Send the Excel file to the specified reports channel
        # reports_channel_id = int(os.getenv('TIMECARD_REPORTS_CHANNEL_ID'))
        # reports_channel = self.bot.get_channel(reports_channel_id)
        # if reports_channel:
        #     await reports_channel.send(file=discord.File(file_path))
        print(f"Weekly report for the week ending on {week_end_date} has been generated and sent to the reports channel.")
        # else:
        #     print("Reports channel not found.")
    except ValueError:
        print("Invalid date format. Please use YYYY-MM-DD.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [133]:
weekly_report(db=f"{os.getcwd()}\\timetracker.db",week_end_date="2024-07-06",employee_group="Hipp Temporary Skills, Inc.")

C:\Users\Chromes\AppData\Local\Temp\ipykernel_15968\2585595283.py:59: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute(sql_query, params)


Weekly report for the week ending on 2024-07-06 has been generated and sent to the reports channel.


In [93]:
## Test to show that the sorting algorithm works and it sorts by the punch_in time 
dakota_def = [(('Employee 12', 13, '2024-07-01 07:30:00', '2024-07-01 18:08:00', 'TRUE', 'TRUE', 0), [('Construction', 'Gerhart', 525)]), (('Employee 12', 27, '2024-07-02 07:34:00', '2024-07-02 17:39:00', 'TRUE', 'TRUE', 0), [('Construction', 'Gerhart', 240), ('Construction', 'Chumbley', 180)]), (('Employee 12', 37, '2024-07-03 07:46:00', '2024-07-03 16:14:00', 'TRUE', 'TRUE', 0), [('Construction', 'Gerhart', 120), ('Construction', 'Chumbley', 75), ('Service', 'Feroz', 150)]), (('Employee 12', 38, '2024-07-03 07:47:00', '2024-07-03 16:14:00', 'TRUE', 'TRUE', 0), [('Construction', 'Gerhart', 120), ('Construction', 'Chumbley', 75), ('Service', 'Feroz', 150)])]
print("Default Data")
for dat1, dat2 in dakota_def: #data is a list of tuples that has dat1 
    print(f"    {dat1}")   #which has a tuple of all the information 
    print(f"        {dat2}") #for the shift punch and dat2 is a list of workpunches for the shift
print("Altered Data")
dakota = [(('Employee 12', 27, '2024-07-02 07:34:00', '2024-07-02 17:39:00', 'TRUE', 'TRUE', 0), [('Construction', 'Gerhart', 240), ('Construction', 'Chumbley', 180)]), (('Employee 12', 13, '2024-07-01 07:30:00', '2024-07-01 18:08:00', 'TRUE', 'TRUE', 0), [('Construction', 'Gerhart', 525)]), (('Employee 12', 38, '2024-07-03 07:47:00', '2024-07-03 16:14:00', 'TRUE', 'TRUE', 0), [('Construction', 'Gerhart', 120), ('Construction', 'Chumbley', 75), ('Service', 'Feroz', 150)]), (('Employee 12', 37, '2024-07-03 07:46:00', '2024-07-03 16:14:00', 'TRUE', 'TRUE', 0), [('Construction', 'Gerhart', 120), ('Construction', 'Chumbley', 75), ('Service', 'Feroz', 150)])]
for dat1, dat2 in dakota: #data is a list of tuples that has dat1 
    print(f"    {dat1}")   #which has a tuple of all the information 
    print(f"        {dat2}") #for the shift punch and dat2 is a list of workpunches for the shift
dakota.sort(key=getKey)
print("Sorted Data")
for dat1, dat2 in dakota: #data is a list of tuples that has dat1 
    print(f"    {dat1}")   #which has a tuple of all the information 
    print(f"        {dat2}") #for the shift punch and dat2 is a list of workpunches for the shift
dakota_def == dakota

Default Data
    ('Employee 12', 13, '2024-07-01 07:30:00', '2024-07-01 18:08:00', 'TRUE', 'TRUE', 0)
        [('Construction', 'Gerhart', 525)]
    ('Employee 12', 27, '2024-07-02 07:34:00', '2024-07-02 17:39:00', 'TRUE', 'TRUE', 0)
        [('Construction', 'Gerhart', 240), ('Construction', 'Chumbley', 180)]
    ('Employee 12', 37, '2024-07-03 07:46:00', '2024-07-03 16:14:00', 'TRUE', 'TRUE', 0)
        [('Construction', 'Gerhart', 120), ('Construction', 'Chumbley', 75), ('Service', 'Feroz', 150)]
    ('Employee 12', 38, '2024-07-03 07:47:00', '2024-07-03 16:14:00', 'TRUE', 'TRUE', 0)
        [('Construction', 'Gerhart', 120), ('Construction', 'Chumbley', 75), ('Service', 'Feroz', 150)]
Altered Data
    ('Employee 12', 27, '2024-07-02 07:34:00', '2024-07-02 17:39:00', 'TRUE', 'TRUE', 0)
        [('Construction', 'Gerhart', 240), ('Construction', 'Chumbley', 180)]
    ('Employee 12', 13, '2024-07-01 07:30:00', '2024-07-01 18:08:00', 'TRUE', 'TRUE', 0)
        [('Construction', 'Gerhar

True

In [94]:
datetime.strptime("2024-07-8 07:47:00", '%Y-%m-%d %H:%M:%S').weekday()

0